#Clone Repository

In [9]:
!rm -rf *
!rm -rf .git* .config
!ls -la

total 12
drwxr-xr-x 1 root root 4096 May 30 06:05 .
drwxr-xr-x 1 root root 4096 May 30 05:37 ..


In [24]:
!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/manuhg/yolo_retrain
   7e91d76..4634317  master     -> origin/master
Updating 7e91d76..4634317
Fast-forward
 Readme.md |  9 +++++++++
 train.py  | 34 +++++++++++++++++++++++++++++++++-
 2 files changed, 42 insertions(+), 1 deletion(-)
 create mode 100644 Readme.md


# Do it at once

In [11]:
!python train.py -d . -ts Pascal_VOC -m yolov2-tiny

Cloning darknet repository
Cloning into 'dkn'...
remote: Enumerating objects: 5901, done.
remote: Total 5901 (delta 0), reused 0 (delta 0), pack-reused 5901
Receiving objects: 100% (5901/5901), 6.16 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (3921/3921), done.
git clone https://github.com/pjreddie/darknet.git dkn 
mv -v dkn/* ./ renamed 'dkn/LICENSE' -> './LICENSE'
renamed 'dkn/LICENSE.fuck' -> './LICENSE.fuck'
renamed 'dkn/LICENSE.gen' -> './LICENSE.gen'
renamed 'dkn/LICENSE.gpl' -> './LICENSE.gpl'
renamed 'dkn/LICENSE.meta' -> './LICENSE.meta'
renamed 'dkn/LICENSE.mit' -> './LICENSE.mit'
renamed 'dkn/LICENSE.v1' -> './LICENSE.v1'
renamed 'dkn/Makefile' -> './Makefile'
renamed 'dkn/README.md' -> './README.md'
renamed 'dkn/cfg' -> './cfg'
renamed 'dkn/data' -> './data'
renamed 'dkn/examples' -> './examples'
renamed 'dkn/include' -> './include'
renamed 'dkn/python' -> './python'
renamed 'dkn/scripts' -> './scripts'
renamed 'dkn/src' -> './src'

sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDN

KeyboardInterrupt: ignored

 # Do it step by step

## Get  Pascal VOC dataset

In [0]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

!cat 2007_train.txt 2012_*.txt > train.txt

!cp 2007_val.txt test.txt

!cp data/voc.names classes.txt

!mkdir bkup
!mv 2* bkup/
!mv *.tar bkup/

## Get NFPA dataset

In [0]:
!pip install gdown

In [0]:
!gdown https://drive.google.com/uc?id=1_gJcci9p6cS0EucumFt2gpwA0tqgP7_f&export=download

In [0]:
!unzip NFPA_dataset.zip

In [0]:
!mkdir nfpa

In [0]:
!mv 'NFPA dataset' nfpa/data

In [0]:
!cp nfpa/data/train.txt ./
!cp nfpa/data/test.txt ./
!printf "NFPA" > classes.txt

## Train

### Prepare config

In [0]:
data_dir = '.'
class_names_file = 'classes.txt'
batch_size, subdivisions = int('64'), int('8')
model_name='yolov2'
model_url='https://pjreddie.com/media/files/darknet19_448.conv.23'
filename='yolo_custom.cfg'
train_sample = 'Pascal_VOC'
    
import os
import sys
import subprocess
from gen_files import gen
import argparse
from utils import exec_cmd,get_NFPA_dataset,get_PASCAL_VOC_dataset


get_datasets={'NFPA':get_NFPA_dataset,'Pascal_VOC':get_PASCAL_VOC_dataset}
dataset_dw_func = get_datasets.get(train_sample)

In [4]:
if not os.path.isfile('./darknet'):
    print('Cloning darknet repository')
    exec_cmd('git clone https://github.com/pjreddie/darknet.git dkn')
    exec_cmd('mv -v dkn/* ./')
    exec_cmd("sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDNN=1/;s/OPENCV=0/OPENCV=1/;s/OPENMP=0/OPENMP=1/;' Makefile")
    exec_cmd('make -j8')

if not os.path.isfile('darknet19_448.conv.23'):
    print('Downloading darknet model')
    exec_cmd('wget '+model_url)

Cloning darknet repository
git clone https://github.com/pjreddie/darknet.git dkn 
mv -v dkn/* ./ renamed 'dkn/LICENSE' -> './LICENSE'
renamed 'dkn/LICENSE.fuck' -> './LICENSE.fuck'
renamed 'dkn/LICENSE.gen' -> './LICENSE.gen'
renamed 'dkn/LICENSE.gpl' -> './LICENSE.gpl'
renamed 'dkn/LICENSE.meta' -> './LICENSE.meta'
renamed 'dkn/LICENSE.mit' -> './LICENSE.mit'
renamed 'dkn/LICENSE.v1' -> './LICENSE.v1'
renamed 'dkn/Makefile' -> './Makefile'
renamed 'dkn/README.md' -> './README.md'
renamed 'dkn/cfg' -> './cfg'
renamed 'dkn/data' -> './data'
renamed 'dkn/examples' -> './examples'
renamed 'dkn/include' -> './include'
renamed 'dkn/python' -> './python'
renamed 'dkn/scripts' -> './scripts'
renamed 'dkn/src' -> './src'

sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDNN=1/;s/OPENCV=0/OPENCV=1/;s/OPENMP=0/OPENMP=1/;' Makefile 
make -j8 mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-resu

In [5]:
if dataset_dw_func is not None:
    print('Downloading dataset')
    dataset_dw_func()

wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar 
wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar 
wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar 
tar xf VOCtrainval_11-May-2012.tar 
tar xf VOCtrainval_06-Nov-2007.tar 
tar xf VOCtest_06-Nov-2007.tar 
wget https://pjreddie.com/media/files/voc_label.py 
python voc_label.py 
cat 2007_train.txt 2012_*.txt > train.txt 
cp 2007_val.txt test.txt 
cp data/voc.names classes.txt 
mkdir bkup 
mv 2* bkup/ 
mv *.tar bkup/ 


In [6]:
with open(data_dir+'/'+class_names_file) as f:
            class_names = list(
                map(lambda s: s.replace('\n', '').strip(), f.readlines()))
            class_names = list(filter(None,class_names))

data_file, names_file, cfg_file = gen(
    class_names, model_name=model_name, batch_size=batch_size, subdivisions=subdivisions, filename=filename)
flag = True
print(data_file, names_file, cfg_file)

Configuring to train  yolov2  classes: 20
cp cfgs/yolov2.cfg cfg/yolo_custom.cfg 
sed -i '6s/.*/batch=64/;7s/.*/subdivisions=8/;237s/.*/filters=125/;244s/.*/classes=20/;' 'cfg/yolo_custom.cfg' 
cfg/obj.data cfg/obj.names cfg/yolo_custom.cfg


###  Train the model

#### sub process

In [0]:
print('Traning the model')
cmd = './darknet detector train '+data_file +' '+cfg_file+' darknet19_448.conv.23'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1,shell = True)
for line in iter(p.stdout.readline,''):
    print(line)
p.stdout.close()
p.wait()

#### Direct commandline

In [7]:
!./darknet detector train cfg/obj.data cfg/yolo_custom.cfg darknet19_448.conv.23

yolo_custom
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 max          2 x 2 / 2   608 x 608 x  32   ->   304 x 304 x  32
    2 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    3 max          2 x 2 / 2   304 x 304 x  64   ->   152 x 152 x  64
    4 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    5 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    6 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    7 max          2 x 2 / 2   152 x 152 x 128   ->    76 x  76 x 128
    8 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256  3.407 BFLOPs
    9 conv    128  1 x 1 / 1    76 x  76 x 256   ->    76 x  76 x 128  0.379 BFLOPs
   10 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256  3.407 BFLOPs
   11 max          2 x 2 / 2    76 x  7

# Inference

In [5]:
# !python train.py -d . -ts Pascal_VOC -m yolov2-tiny -ri yes
!./darknet detect cfg/yolo_custom.cfg backup/yolo_custom_900.weights /content/VOCdevkit/VOC2007/JPEGImages/000024.jpg

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

In [10]:
!git clone https://github.com/manuhg/yolo_retrain.git .

Cloning into '.'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 67 (delta 31), reused 59 (delta 23), pack-reused 0
Unpacking objects: 100% (67/67), done.
